**Transfer learning in image classification**

In this notebook transfer learning is used pre-trained model from google's Tensorflow Hub is taken and re-trained that on flowers dataset. 

Using pre-trained model saves lot of time and computational budget for new classification problem at hand

In [1]:
import numpy as np
import cv2

import PIL.Image as Image
import os

import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [3]:
IMAGE_SHAPE = (224, 224)
classifier = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4" , input_shape = IMAGE_SHAPE + (3,))
])

In [5]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1001)              3540265   
                                                                 
Total params: 3,540,265
Trainable params: 0
Non-trainable params: 3,540,265
_________________________________________________________________


### LOAD DATASET

In [6]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)
# cache_dir indicates where to download data. I specified . which means current directory
# untar true will unzip it

228813984/228813984 [==============================] - 1s 0us/step


In [7]:
data_dir

'./datasets/flower_photos'

In [8]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('datasets/flower_photos')

In [9]:
list(data_dir.glob('*/*.jpg'))[:5]

[PosixPath('datasets/flower_photos/tulips/16139439153_fbdee29a10_n.jpg'),
 PosixPath('datasets/flower_photos/tulips/7166570828_7c26ca5766_n.jpg'),
 PosixPath('datasets/flower_photos/tulips/5674127693_1ddbd81097.jpg'),
 PosixPath('datasets/flower_photos/tulips/4300258119_b03f2f956e.jpg'),
 PosixPath('datasets/flower_photos/tulips/2426849837_baefd9a518_n.jpg')]

In [10]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

3670


### Read flowers images from disk into numpy array using opencv

In [11]:
flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

In [12]:
flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

### Create X and y i.e independent and dependent variables

In [13]:
X, y = [], []

for flower_name, images in flowers_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(224,224))
        X.append(resized_img)
        y.append(flowers_labels_dict[flower_name])

In [14]:
X = np.array(X)
y = np.array(y)

### Train test split

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### Preprocessing: scale images

In [16]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

### Now take pre-trained model and retrain it using flowers images

**Feature Vector** gives all layers other than the last layer

In [17]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [18]:
num_of_flowers = 5

model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(num_of_flowers)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [19]:
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

model.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
86/86 [==============================] - 13s 41ms/step - loss: 0.8886 - acc: 0.6668
Epoch 2/5
86/86 [==============================] - 4s 41ms/step - loss: 0.4165 - acc: 0.8608
Epoch 3/5
86/86 [==============================] - 4s 41ms/step - loss: 0.3239 - acc: 0.8964
Epoch 4/5
86/86 [==============================] - 4s 41ms/step - loss: 0.2672 - acc: 0.9201
Epoch 5/5
86/86 [==============================] - 4s 41ms/step - loss: 0.2296 - acc: 0.9360


In [20]:
model.evaluate(X_test_scaled,y_test)

29/29 [==============================] - 2s 54ms/step - loss: 0.4030 - acc: 0.8519


[0.4030470550060272, 0.8518518805503845]

In [21]:
from sklearn.metrics import confusion_matrix , classification_report
import numpy as np
y_pred = model.predict(X_test_scaled)
y_pred_classes = [np.argmax(element) for element in y_pred]

print("Classification Report: \n", classification_report(y_test, y_pred_classes))

29/29 [==============================] - 2s 40ms/step
Classification Report: 
               precision    recall  f1-score   support

           0       0.84      0.84      0.84       176
           1       0.90      0.84      0.87       154
           2       0.89      0.92      0.90       226
           3       0.75      0.88      0.81       150
           4       0.86      0.78      0.82       212

    accuracy                           0.85       918
   macro avg       0.85      0.85      0.85       918
weighted avg       0.86      0.85      0.85       918

